# Obtain data of QSO from SIMBAD, filtering the objects with data of [Ne II], [Ne III] and [Ne V] in infrared from NED Phot Tables and saving it

In [2]:
# If this notebook will be run in Google Colab, first at all the next code must be run. If the notebook will be run in Jupyter
#then "comment" or omit this part 

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery

In [3]:
from datetime import date
import time
import requests
import numpy as np

import pyvo
from astroquery.ipac.ned import Ned
from astroquery.simbad import Simbad

from pathlib import Path
from requests import Request, Session

import astropy.units as u
from astropy.table import hstack, QTable, Table
from astropy.coordinates import SkyCoord

import getphotNED as pN

In [4]:
path_raw='../Datos/Raw/'
Path(path_raw).mkdir(parents=True, exist_ok=True)
path_phot_ned = '../Datos/Work/Phot_Tables_NED/' #In this path wi will save the photometry tables for each object
Path(path_phot_ned).mkdir(parents=True, exist_ok=True)

In [5]:
service = pyvo.dal.TAPService("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

result = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,ids.ids
    ,alltypes.otypes
FROM basic
JOIN ids ON oid = ids.oidref
JOIN alltypes ON oid = alltypes.oidref
WHERE basic.otype = 'QSO'
""")

In [6]:
simbad_sample_QSO = result.to_table()
obj_name_SMB=simbad_sample_QSO["main_id"]
obj_ids_SMB=simbad_sample_QSO["ids"]
len(simbad_sample_QSO)

50000

In [7]:
today = date.today()
today.strftime('%b_%d_%Y')

'Sep_04_2022'

In [8]:
INIT_URL = 'http://vizier.u-strasbg.fr/viz-bin/sed?-c='
urls = [INIT_URL+str(simbad_sample_QSO['ra'][row])+'%20'+
        str(simbad_sample_QSO['dec'][row])+'&-c.rs=5' for row in range(len(simbad_sample_QSO))]
simbad_sample_QSO['cds_url'] = urls

In [10]:
idS = ['QSO B1443+0957'] #Id to search
idx = [np.where(simbad_sample_QSO['main_id'] == idt)[0] for idt in idS]
print(obj_name_SMB[idx[0]])
print('SMB sample index: ',idx[0])

main_id
-------
SMB sample index:  []


In [11]:
simbad_sample_QSO.write(path_raw+'SMB_'+today.strftime('%b_%d_%Y')+'_QSO_Samples'+'.vot',
                    format='votable',overwrite=True)

cat_smb_sample = SkyCoord(ra=simbad_sample_QSO['ra'], dec=simbad_sample_QSO['dec'])

Ne_IR_Fq = np.array([19200000000000.0,19300000000000.0,20900000000000.0,21000000000000.0,23400000000000.0])*u.Hz

In [12]:
def FilterbyNeFreq(NEDTable, Freq):
    
    '''
    This function is for search in a photometry
    table is there is information of [Ne] lines
    based to a specific frequency values and 
    return the which of those frequencies was founded'''
    
    NEDTable.remove_rows(np.where(NEDTable['Flux Density'].mask)[0])
    NEDTable.remove_rows(np.where(NEDTable['NED Units'] == 'Jy')[0])
    #NEDTable.remove_rows(np.where(NEDTable['NED Uncertainty'] == '')[0])
    #NEDTable.remove_rows(np.where(NEDTable['NED Uncertainty'] == '+/-...')[0])
    NEDTable.remove_rows(np.where(NEDTable['Observed Passband'] == '[Cl II] line (IRS)')[0])
    NEDTable.remove_rows(np.where(NEDTable['Observed Passband'] == '[Cl II] 14.4 (IRS)')[0])
    NEDTable.remove_rows(np.where(NEDTable['Observed Passband'] == '[Cl II] (IRS)')[0])
    NEDTable.remove_rows(np.where(NEDTable['Observed Passband'] == '[Cl II] 14.37 (IRS)')[0])
    
    f = np.array([NEDTable['Frequency']])*u.Hz #We get the Frequency column from the table
    intersec = np.intersect1d(Freq, f) #Intersection between Fr and Ne_IR_Fq
   
    return intersec

In [ ]:
NeIR_obj_t = QTable(names=('Main_Id','Id_used_NED','RA','DEC','otype_txt','redshift',
                           'rvz_type','ids','otypes','cds_url','SMB_sample_index'),
                    dtype=('O','O','f8','f8','O','f8','U1','O','O','U93','f8'))

Ne_Inf = []

print(f"{'#':4} {'Idx':6} {'Name ID':^27} {'ID used in NED':^27} {'Ne IR Info':^20}")
i=-1
start = time.time()
for ind, id_smb in enumerate(obj_name_SMB):
    try:       
        print('\nIdx SIMBAD: ',ind,' main_id: ',id_smb)
        phot_T,name_NED = pN.photNED(id_smb,obj_ids_SMB[ind],cat_smb_sample[ind])
        #Save table again for work with it before save the original one
        phot_t,name_ned = pN.photNED(id_smb,obj_ids_SMB[ind],cat_smb_sample[ind])
        ins = FilterbyNeFreq(phot_t, Ne_IR_Fq) #Search for the [Ne] frequencies in NED table
        print(name_ned)
        
        if len(ins)>0: #First filter: If the phot table has infomation of frequencies of Fq            
            #print(ins)
            i=i+1
            phot_T.write(path_phot_ned+id_smb+'_NED_phot_tables'+'.vot', #Save the phot table in a vot table file
                            format='votable',overwrite=True)
            NeIR_obj_row = [id_smb,name_ned,simbad_sample_Sy["ra"][ind],simbad_sample_Sy["dec"][ind],
                            simbad_sample_Sy["otype_txt"][ind],simbad_sample_Sy["rvz_redshift"][ind],
                            simbad_sample_Sy["rvz_type"][ind],simbad_sample_Sy["ids"][ind],
                            simbad_sample_Sy["otypes"][ind],simbad_sample_Sy["cds_url"][ind],ind] 
            NeIR_obj_t.add_row(vals=NeIR_obj_row)
            
            if (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins)and (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins)\
            and Ne_IR_Fq[4] in ins: #Second filter to classify according the Ne info after remove empty flux values
                Ne_info = 'NeII|NeIII|NeV'
            elif (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins) and Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII|NeIII'
            elif (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins) and Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII|NeV'
            elif (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins) and (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins):
                Ne_info = 'NeIII|NeV'
            elif Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins:
                Ne_info = 'NeIII'
            elif Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins:
                Ne_info = 'NeV'
            elif Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII'
            else:
                Ne_info = 'No Flux Values' 
            Ne_Inf.append(Ne_info)
            print(f"{'%g'%i:4} {'%g'%ind:6} {'%s'%id_smb:^27} {'%s'%name_ned:^27} {'%s'%Ne_info:^20}")
    
    except KeyError:
        pass

end = time.time()

print('\nTotal objects with lines emission of NeII or NeIII or NeV in IR  found: ',len(NeIR_obj_t))
print('Execution Time(seg): ' + str(end - start))
print('Execution Time(min): ' + str((end - start)/60))
print('Execution Time(hrs): ' + str((end - start)/3600))

#    Idx              Name ID                 ID used in NED             Ne IR Info     

Idx SIMBAD:  0  main_id:  SDSS J150432.30+301658.3
SDSS J150432.30+301658.3

Idx SIMBAD:  1  main_id:  SDSS J143709.36+365949.2
SDSS J143709.36+365949.2

Idx SIMBAD:  2  main_id:  SDSS J100551.12+535429.2
       Object Name       
-------------------------
WISEA J100551.14+535429.2

Idx SIMBAD:  3  main_id:  SDSS J120320.13+531336.9
SDSS J120320.13+531336.9

Idx SIMBAD:  4  main_id:  SDSS J120320.06+324404.3
       Object Name       
-------------------------
WISEA J120320.06+324405.1

Idx SIMBAD:  5  main_id:  SDSS J120320.37+464226.7
SDSS J120320.37+464226.7

Idx SIMBAD:  6  main_id:  SDSS J143709.36+575040.6
SDSS J143709.36+575040.6

Idx SIMBAD:  7  main_id:  SDSS J150432.34+562833.8
SDSS J150432.34+562833.8

Idx SIMBAD:  8  main_id:  SDSS J120320.18+100114.3
SDSS J120320.18+100114.3

Idx SIMBAD:  9  main_id:  SDSS J100551.14+000825.6
SDSS J100551.14+000825.6

Idx SIMBAD:  10  main_id:  [VV2006

      Object Name       
------------------------
SDSS J120321.66+123848.6

Idx SIMBAD:  91  main_id:  SDSS J120321.66+631307.6
SDSS J120321.66+631307.6

Idx SIMBAD:  92  main_id:  SDSS J143710.31+274009.5
SDSS J143710.31+274009.5

Idx SIMBAD:  93  main_id:  SDSS J150433.11+573403.6
       Object Name       
-------------------------
WISEA J150433.06+573403.3

Idx SIMBAD:  94  main_id:  SDSS J122352.21+491145.3
SDSS J122352.21+491145.3

Idx SIMBAD:  95  main_id:  SDSS J143123.37+433345.0
SDSS J143123.37+433345.0

Idx SIMBAD:  96  main_id:  SDSS J143710.36+444425.6
SDSS J143710.36+444425.6

Idx SIMBAD:  97  main_id:  SDSS J140842.83+471508.9
       Object Name       
-------------------------
WISEA J140842.83+471508.9

Idx SIMBAD:  98  main_id:  [VV2006] J120321.9+041419
BWE 1200+0430

Idx SIMBAD:  99  main_id:  SDSS J150433.21+082657.7
SDSS J150433.21+082657.7

Idx SIMBAD:  100  main_id:  SDSS J100553.79+481142.1
SDSS J100553.79+481142.1

Idx SIMBAD:  101  main_id:  SDSS J100553.76+443

In [ ]:
NeIR_obj_t['NED_Ne_IR_info'] = Ne_Inf #Add a column to save the Ne info
NeIR_obj_t.show_in_notebook()

In [ ]:
NeIR_obj_t.write(path_raw+'Obj_QSO_Ne-IR_'+today.strftime('%b_%d_%Y')+'.vot',
                    format='votable',overwrite=True)

In [ ]:
NeII_NeIII_NeV_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeII|NeIII|NeV')[0] #List of objects with NeII, NeIII and NeV data
NeII_NeIII_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeII|NeIII')[0] #List of objects with NeII and NeIII data
NeII_NeV_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeII|NeV')[0] #List of objects with NeII and NeV data
NeIII_NeV_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeIII|NeV')[0] #List of objects with NeIII and NeV data
NeIII_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeIII')[0] #List of objects with NeIII data
NeV_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeV')[0] #List of objects with NeV data
NeII_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeII')[0] #List of objects with NeII data
No_Ne_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'No Flux Values')[0] #List of objects with NeII data
print('Total objects found with NeII, NeIII and NeV emission lines: ',len(NeII_NeIII_NeV_IR))
print('Total objects found with NeII and NeIII emission lines: ',len(NeII_NeIII_IR))
print('Total objects found with NeII and NeV emission lines: ',len(NeII_NeV_IR))
print('Total objects found with NeIII and NeV emission lines: ',len(NeIII_NeV_IR))
print('Total objects found with NeIII emission lines: ',len(NeIII_IR))
print('Total objects found with NeV emission lines: ',len(NeV_IR))
print('Total objects found with NeII emission lines: ',len(NeII_IR))
print('Total objects found with No Ne flux values: ',len(No_Ne_IR))

In [ ]:
%reload_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy